We will need some source code to work with later so lets start the process of checking out an LFRic core working copy:

In [ ]:
fcm co --quiet https://code.metoffice.gov.uk/svn/lfric/LFRic/trunk $ROOT/core-trunk

And one for LFRic applications:

In [ ]:
fcm co --quiet https://code.metoffice.gov.uk/svn/lfric_apps/main/trunk $ROOT/apps-trunk

# Building LFRic Software

A rough guide to using the LFRic build system.

## Legal Obligation

All presentations regarding LFRic software must, by law, contain a diagram summarising the PSyKAl
approach to separating scientific and technical software:

![Science developer writes an application consisting algorithms which work on whole fields and kernels
which work on field cells. Between these is the Parallel System (Psy) layer which is generated by the
PSyclone tool and manages optimisation.](PSyKAl.svg)

## Building LFRic Core

In this section we will learn how to build LFRic core software.

### Prerequisites

Versions are those currently used at the Met Office. They are not magic but there
are some requirements.

#### Libraries

* MPI([MPICH](https://www.mpich.org/) 3.4.1)
* [HDF5](https://www.hdfgroup.org/solutions/hdf5) 1.12.1
* [NetCDF](https://www.unidata.ucar.edu/software/netcdf) 4.8.1
* [YAXT](https://dkrz-sw.gitlab-pages.dkrz.de/yaxt/index.html) 0.9.2.1
* [XIOS](https://forge.ipsl.jussieu.fr/ioserver/wiki) (SSL certificate error) r2252 (minimum r2252)
* [pFUnit](https://github.com/Goddard-Fortran-Ecosystem/pFUnit) 4.10.0 (minimum 4.0.0)

#### Tools

* GNU [Make](https://www.gnu.org/software/make) 3.82 (minimum 3.82)
* Fortran Compiler
* [Python](https://www.python.org) 3.7.0
  * [Jinja2](https://github.com/pallets/jinja) 3.0.3
  * [fParser](https://github.com/stfc/fparser) 0.1.4
* [PSyclone](https://github.com/stfc/psyclone) 3.0.0 (minimum 3.0.0)
* RosePicker (LFRic Core repository) 2.0.0 (minimum 2.0.0)

In [ ]:
module load environment/lfric/ifort
module list

Among these prerequisites note `RosePicker`. This is a utility for reading Rose configuration metadata
files and converting them to an intermediate format. This is necessary for licencing reasons and may be
found in the LFRic core repository.

### Developer Environment

The LFRic build system makes use of de-facto standard environment variables to understand how to use
the platform it finds itself on.

* `PATH` - Search paths for executables, colon separated.
* `FPP` - Fortran preprocessor
* `FC` - Fortran compiler
* `FFLAGS` - arguments passed to the Fortran compiler, space separated
  * `-I` - Search path for modules
* `LDMPI` - MPI Wrapper to use for linking (e.g. `mpif90`) (Non-standard)
* `LDFLAGS` - Arguments passed to the linker, space separated
  * `-L` - Search path for libraries used in linking
* `LD_LIBRARY_PATH` - Search paths for libraries used at runtime, colon separated

If your development platform requires specific compiler arguments in addition to
those normally needed to build they can be added to these environment variables.

The Met Office makes use of an environment module system to manage all this but
they are, fundamentally, just fancy shell scripts so you could use shell scripts
or any other appropiate means instead.

In [ ]:
echo PATH : $PATH
echo FPP : $FPP
echo FC : $FC
echo FFLAGS : $FFLAGS
echo LDMPI : $LDMPI
echo LDFLAGS : $LDFLAGS
echo LD_LIBRARY_PATH : $LD_LIBRARY_PATH

### Building

The process should be quite straight forward once a suitible development environment
is in place but first a little about how building works at the conceptual level

#### Phases of a Build

In order to simplify matters all the materials needed for a compile are martialed
into a working space before the compile is undertaken. This is a multi-stage process
itself and includes:

1. Generate source from templates
1. Copy Fortran source
1. Use PSyclone with algorithm (`.[Xx]90`) files to generate PSy layer

These will happen for each component the current project makes use of, so they are likely
to appear at least twice since everything uses the infrastructure except the infrastructure.

4. Use "configurator" on `rose_meta.conf` file to generate namelist loaders
1. Generate unit tests using pFUnit framework

With this done it is then possible to analyse every file in the work space for dependency
information, then generate a dependency tree. This will allow the build to occur in the
correct order and exclude any unused files.

With all that done the actual compilation is undertaken.

#### Performing a Build

In [ ]:
cd $ROOT/core-trunk/mesh_tools
make -j

The default target will build any executables as well as building and running
unit and integration tests. This is considered the most useful mode for development
as it provides constant and automatic testing.

#### Build Targets

The build system offers a number of targets which developers often find useful.

* `clean` to delete the work space and remove any built artefacts
* `build` compile and link only the project executables
* `unit-tests` compile, link and run only the project unit tests
* `integration-tests` compile, link and run only the project integration tests.

Note that build, unit and integration tests are accessible as separate targets and
this can be useful but they should not generally be used on their own. The better
approach is to use the default and always run them together. Remember, just because
an error is not reported does not mean it has gone away.

#### Build Options

Also supported by the build system are a number of options controlled by setting
variables.

If you are interested in what the build system is doing:

```
make VERBOSE=1
```

If you need a statically linked executable:

```
make LINK_TYPE=static
```

This is the default on Cray platforms.

### Has It Worked?

Most projects are provided with a "canned test." This is a minimal example
configuration which provides a simple sanity check on your compile.

In [ ]:
cd example
../bin/cubedsphere_mesh_generator cubedsphere_mesh_gen.nml

In [ ]:
../bin/summarise_ugrid cubedsphere_mesh.nc

Note that it is not intended as a development aid, and certainly shouldn't be extended or otherwise developed. It is only here to confirm you've built something somewhat executable.

## Building LFRic Applications

The applications are built using the same build system but are invoked
from the working copy root using an additional layer of wrapping:

```
build/local_build.py -a <project> [-t <target>]
```

If you wish to use one of the build variables you will have to pass it
as an environment variable:

```
VERBOSE=1 build/local_build.py ...
```

For example:

```
LINK_TYPE=static build/local_build.py -a linear -t unit-tests
```

In [ ]:
cd $ROOT/apps-trunk
build/local_build.py -a socrates_interface -t unit-tests

## Transformation Scripts

Psyclone can make use of so-called "transformation scripts." These are
[pieces of Python scripting](https://psyclone.readthedocs.io/en/stable/transformations.html)
which take advantage of PSyclone library functions to modify the generated source code.

These transformations may be used for a number of reasons but the most
obvious and common is for optimisation.

### Per Platform

Each target platform must have a default transformation script which may be a null
transform. This is held in the file `<project>/optimisation/<platform>/global.py`.

### Per File

If you have special requirements for a specific file then you may utilise a per-file
optimisation script by creating
`<project>/optimisation/<platform>/<algorithm source path>/<filename>.py`.

For example:
`simple_diffusion/optimisation/minimum/algorithm/sci_checksum_alg_mod.py`

Note that the algorithm being transformed here comes from the science component.
Transformation scripts are used at application level, not library level.

### Short-term Hand Editing

It may be useful to experiment with the generated source by making short lived
hand edits.

The generated souce may be found in
`<project>/working/<project>/path/<algorithm>_mod_psy.f90` and may be edited to your
heart's content. Be aware, though, that it will be overwritten the next time the
algorithm source is modified.

### Medium-term Overriding

There are two primary reasons for wanting to override PSyclone generated
code: Bugs and missing features.

#### Working Around Bugs

When PSyclone is unable to help due to a bug it can be necessary to
override the generated code.

These are held in `<project>/source/psy` and are a modified copy of the generated
PSy layer code.

Imagine PSyclone suffered a bug which meant output for one algorithm was
malformed, in that case we could copy the generated source and fix it by hand. The resulting fixed version is placed in `components/science/source/psy/sci_field_minmax_alg_mod_psy.f90`.

#### Stop-gap For Missing Features

In the case where PSyclone does not yet support a requirement it may be
necessary to make a so-called "PsyKAl light" implementation. This
involves writing an appropriate "invoke" subroutine which is usually placed in `<project>/source/psy/psykal_light_mod.f90`.

With this new invokation available it is called directly
(`call invoke_my_thing(...`) rather than with the normal method,
`call invoke(my_thing_type(...`.

Any such "light" implementation must be acompanied by the PSyclone issue relating to the missing feature. This helps us work out which "light" code can be retired as issues are closed.

## Supporting New Compilers

If you want to take advantage of a compiler which is not currently supported
you will need to add a Make fragment file to set up some variables which will
be used by the rest of the build system.

Refer to the existing files for examples.

### Fortran Compilers

A new Fortran compiler is described in `infrastructure/build/fortran/<compilere exe>.mk`. 

It is useful to have the compiler version available in a variable `<COMPILER>_VERSION`
so bug work-arounds can be isolated to the versions they effect.

Beyond that the following variables must be defined:

| Variable                    | Example                | Purpose                                       |
|-----------------------------|------------------------|-----------------------------------------------|
| `F_MOD_DESTINATION_ARG`     | `-module$(SPACE)`      | Argument used to control where module files are placed. Note the use of `$(SPACE)` to force a space character, otherwise it is liable to get lost. |
| `F_MOD_SOURCE_ARG`          | `-I`                   | Argument used to control where module files a sought. |
| `FFLAGS_OPENMP`             | `-qopenmp`             | Arguments to switch on OpenMP support.         |
| `FFLAGS_NO_OPTIMISATION`    | `-O0`                  | Arguments oo turn off optimisation.            |
| `FFLAGS_SAFE_OPTIMISATION`  | `-O2 -fp-model strict` | Arguments to turn on optimisation which does not effect debugging and doesn't change floating point results. |
| `FFLAGS_RISKY_OPTIMISATION` | `-O3 -xhost`           | Arguments to turn on maximum optimisation which may change floating point results. |
| `FFLAGS_DEBUG`              | `-g -traceback`        | Arguments to turn on debugging support such as debug symbols. |
| `FFLAGS_WARNINGS`           | `-warn all -warn errors` | Arguments to enable warnings.                |
| `FFLAGS_UNIT_WARNINGS`      | `-warn all`            | Arguments to enable warnings for unit testing. |
| `FFLAGS_INIT`               | `-ftrapub`             | Arguments to enable run-time variable initialisation. |
| `FFLAGS_RUNTIME`            | `-check all -fpe0`     | Arguments to enable run-time checking.         |
| `FFLAGS_FORTRAN_STANDARD`   | `-stand f08`           | Arguments to select language standard.         |
| `FFLAGS_COMPILER`           | `-assume realloc-lhs`  | Arguments used with this compiler - often bug work-arounds. |
| `LDFLAGS_COMPILER`          | ``                     | Linker arguments used with this compiler - usually empty. |
| `FPPFLAGS`                  | `-P`                   | Arguments passed to preprocessor.              |

### C++ Compilers

A new C++ compiler is described in `infrastructure/build/cxx/<compiler exe>.mk`.

As there is no C++ code in any of the LFRic projects there is much less which needs
to be defined about it. The most commonly defined variable is `CXX_RUNTIME_LIBRARY`
which defines the C++ run-time library and is used when linking in order to support
our prerequisites which are C++ libraries.

## Rose/Cylc Suite

The Rose/Cylc suite found in core and applications does build the source but
it is not used for that. They are test suites used to perform system testing and
as such need compiled executables. They are part of the development process only in that they are used for testing.
